

# <center>Projeto LoP DataScience




### Foi projetado para ter 4 tipos de consultas

* A primeira é para receber dados dos professores que estão resgistrados no sistema

* A segunda é para receber dados das turmas que estão registradas no sistema

* A terceira é para receber dados das submissões que estão registradas no sistema

* A quarta é para receber dados das questões que estão registradas no sistema

As consultas vão ser tornando acumulativas, com a segunda dependendo de dados da primeira, e a terceira dependendo de dados da primeira e segunda para o preenchimento da url de consulta. Todas as consultas dependem da key de acesso

Na consulta da class possui as depedências de: 
1. id_teacher, que é o id de um professor
2. year, que é o ano da turma 
3. semester, que é o semestre (1 ou 2) da turma

E na consulta de submission necessita do:
1. id_class, que é o id de uma turma.

Já a consulta de question e do teacher necessita apenas da key

As consultas serão feitas via protocolo HTTP usando o método Get e irão receber um JSON dos dados do LoP


### Estrutra das funções e suas dependências

Funções de coleta e transformações de dados 

- lop_class(), lop_question()
--- lop_submission()
------ question_data()
--------- performance_list()
------------ graph_more_less_70_list_class()
------------ graph_distribution_notes_list()

Funções de treinamento de modelos de Machine Learning

### Descrição dos Dados
#### <center> Dataframe Submissões
| **Semestre** | **Número de atributos (colunas)** | **Número de Registros (linhas)** |
| --- | --- | --- |
| 2020.5 | xxxx | yyy |
| 2020.6 | yyyy | xxx |

#### <center>Dataframe Resultados em LOP
| **Semestre** | **Número de atributos (colunas)** | **Número de Registros (linhas)** |
| --- | --- | --- |
| 2020.5 | xxxx | yyyyy |


---

#### Descrição do datafrane dos professores - Consulta embutida em lop_class()

<b>1. id_teacher</b> ID do professor no LoP

<b>2. email</b> Email do professor

<b>3. name_teacher</b> Nome do professor

--- 

#### Descrição do dataframe das turmas - lop_class()

<b>1. id_teacher</b> ID do professor no LoP

<b>2. id_class</b> ID da turma

<b>3. name_class</b> Nome da turma

<b>4. code</b> Código de acesso a turma no LoP

<b>5. year</b> Ano em que a turma foi cadastrada

<b>6. semester</b> Semestre em que a turma foi cadastrada

---
#### Descrição dos dataframe de Submissão - lop_submission()


<b>1. environment</b> Tipo de máquina

<b>2. hitPercentage</b> Porcentagem de acerto na questão

<b>3. language</b> Linguagem de programação usada

<b>4. char_change_number</b> Número de caracteres alterados

<b>5. timeConsuming</b> Tempo que passou na questão

<b>6. createdAt</b> Data e hora da submissão

<b>7. user</b> Nome e matrícula do aluno

<b>8. question</b> Nome da questão

<b>9. list</b> Nome da lista

<b>10. test</b> Nome da prova

<b>11. id_class</b> ID da turma em que o aluno está associado nessa submissão

---

#### Descrição do dataframe das questões do LoP - lop_question()


<b>1. id_questao</b> ID da questão

<b>2. question</b> Nome da questão

<b>3. difficulty </b> Nível de dificuldade da questão (1-5)

<b>4. tags </b> Esse campo vem com um json que contem os assuntos em que essa questão está associada

<b>5. lists </b> Esse campo vem com um json que contem as listas que a questão está associada 

<b>6. tests</b> Esse campo vem com um json que contem as provas que a questão está associada 

---

#### Descrição do dataframe de dados das questões - question_data()


<b>1. id_list</b> ID da lista

<b>2. list</b> Nome da lista

<b>3. question </b> Nome da questão

<b>4. difficulty </b> Nível de dificuldade da questão (1-5)

<b>5. tags </b> Em cada coluna com nome tag, terá um conteúdo associado a ela

---


#### Descrição do dataframe de performance por lista - performance_list()


<b>1. user</b> Nome e matrícula do aluno

<b>2. list</b> Nome da lista

<b>3. totalHitPercentage </b> Soma de todos acertos de questões únicas

<b>4. totalQuestionsList </b> Número de questões por lista

<b>5. mediaList</b> Esse campo vem com um json que contem as provas que a questão está associada 

---

## 1. Importação das bibliotecas
<a id='importacao'></a>

In [10]:
import pandas as pd
import numpy as np
import json
import urllib3
urllib3.disable_warnings()
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re

## 2. Funções
<a id='funcoes'></a>

Função de leitura de arquivos txt

In [11]:
def read_txt(path):
  with open(path,'r') as file:
    return file.read()

Função de consulta no LoP

In [12]:
def lop_consult(url):
  http = urllib3.PoolManager()
  req = http.request('GET',url)
  if req.status == 200:
    return pd.read_json(req.data, orient = 'RECORDS', encoding = 'utf-8').copy()
  else: 
    return False

Função para obter todas as classes do LoP a partir de uma lista de professores

In [13]:
def lop_class(df_teacher = pd.DataFrame()):
  #Essa função se passar por cópia o df_teacher se torna desnecessário fazer a requisição de todas os professores, e evitando fazer varias consultas
  #Dataframe onde será armazenado todas as turmas
  df_class = pd.DataFrame()
  if df_teacher.empty == True:
    #Formação da url de consulta para saber os professores cadastrados no LoP
    url_teacher = endpoint_teacher + key
    #Realizando a consulta no LoP
    df_teacher = lop_consult(url_teacher)#id, email, name
  #O laço faz uma varredura das turmas de cada professor e por fim armazena em um dataframe de turmas
  for id_teacher in df_teacher['id']:
    #O LoP.v2 foi desenvolvido em 2019, então temos que capturar as turmas entre 2019 e o ano atual
    current_year = datetime.now().year
    for year in range(2019, current_year + 1):
      #Por fim precisamos do semestre que deve ser 1 ou 2
      for semester in range(1,3): 
        try:
          #Url de consulta de classes
          url_class = endpoint_class + id_teacher + '?year=' + str(year) + '&semester=' + str(semester) + '&key=' + key 
          #Dataframe provisório necessário para capturar o tamanho e saber quantas turmas esse professor tem que estar associado
          df_prov = lop_consult(url_class)
          #Inserindo o id do professor no dataframe de turmas
          df_prov['id_teacher'] = id_teacher
          #Adicionando turmas conforme novas consultas
          df_class = df_class.append(df_prov, ignore_index=True).copy()#id, name, code, year, semester, id_teacher
        except:
          pass
  #Renomeando colunas com nomes semelhantes em todos os DF
  df_class.rename(columns={'id':'id_class','name':'name_class'}, inplace = True)
  return df_class

Função para obter todas as submissões a partir de uma lista de turmas

In [14]:
def lop_submission(df_class = pd.DataFrame()):
  #Essa função se passar por cópia o df_class se torna desnecessário fazer a requisição de todas as turmas usando a função lop_class()
  #Dataframe onde serão armazenados todas as submissões 
  df_submission = pd.DataFrame()  
  if df_class.empty == True:
    df_class = lop_class()  
  #Obtem todas as submissões da lista de turmas passadas
  for id_class in df_class['id_class']:
    #Url da consulta as submissões
    url_submission = endpoint_submission + id_class + '/submission?key=' + key
    #Dataframe provisório que vai armazenar o resultado da consulta
    df_prov = lop_consult(url_submission)
    #Inserindo o id_class no dataframe de consultas
    df_prov['id_class'] = id_class
    #Adicionando submissões conforme consultas
    df_submission = df_submission.append(df_prov, ignore_index=True).copy()
  #Campo user trás matricula e o nome do usuário, vamos separar em 2 colunas distintas
  df_submission[['user','registration']] = df_submission['user'].str.split('-',expand=True)
  return df_submission

Função para obter dados das questões cadastradas no LoP

In [15]:
def lop_question():
  url_question = endpoint_question + key
  df_lop_question = lop_consult(url_question)
  df_lop_question.rename(columns = {'id':'id_question','title':'question'}, inplace = True)
  return df_lop_question

Função que transforma os dados das questões extraídas do LoP, assim podemos saber a dificuldade de uma questão, que pode estar associada a uma ou mais listas

In [16]:
def question_data(df_question = pd.DataFrame()):
  #Essa função se passar por cópia o df_question se torna desnecessário fazer a requisição de todas as turmas usando a função lop_question()
  #Dataframes onde serão armazenados todas as questões transformadas
  df_list_data = pd.DataFrame() 
  df_test_data = pd.DataFrame() 
  df_tag_data = pd.DataFrame()  
  df_question_data = pd.DataFrame()  
  #i=0
  if df_question.empty == True:
    df_question = lop_question()
  for question,lists,tests,tags,difficulty in zip(df_question['question'],df_question['lists'],df_question['tests'],df_question['tags'],df_question['difficulty']):
    #Cada registro do json pode ficar com mais de uma lista associada a uma questão, por isso a necessidade de extrair
    df_prov_lists  = pd.DataFrame(json.loads(str(lists).replace("'",'"')))
    #Cada registro do json pode ficar com mais de uma lista associada a uma questão, por isso a necessidade de extrair
    df_prov_tests  = pd.DataFrame(json.loads(str(tests).replace("'",'"')))
    #df_prov_lists.rename(columns = {'id':'id_list','title':'list'}, inplace = True)
    #Inserindo a questão associada
    df_prov_lists['question'] = question
    df_prov_lists['difficulty'] = difficulty
    df_list_data = df_list_data.append(df_prov_lists, ignore_index = True).copy()
    #print(df_list_data['question'])
    #print("numero de questoes unicas no df ",len(df_list_data['question'].unique()))
    df_prov_tests['question'] = question
    df_prov_tests['difficulty'] = difficulty
    df_test_data = df_test_data.append(df_prov_tests, ignore_index = True).copy()
    #Extraindo as tags, que vão dizer o conteudo associado
    df_prov_tags = pd.DataFrame(json.loads(str(tags).replace("'",'"'))).T
    #Inserindo a questão associada
    df_prov_tags['question'] = question    
    df_tag_data = df_tag_data.append(df_prov_tags, ignore_index = True).copy()
    #print(i,' ------------------  ',question, ' ',difficulty)
    #i+=1
    #if i == 100:
    #  break
  
  df_list_data.rename(columns = {'id':'id_list','title':'list'}, inplace = True)
  df_test_data.rename(columns = {'id':'id_test','title':'test'}, inplace = True)
  #print(len(df_list_data['question'].unique()))
  #df_list_data['difficulty'] = df_list_data['difficulty'].astype('int')
  columns_tags = ['question']
  for i in range(len(df_tag_data.columns)-1): columns_tags.append('tag'+str(i+1))
  df_tag_data.columns = columns_tags
  df_list_data = pd.merge(df_list_data, df_tag_data, on = 'question', how = 'outer')#.fillna(0)
  df_test_data = pd.merge(df_test_data, df_tag_data, on = 'question', how = 'outer')#.fillna(0)
  return  pd.concat([df_list_data, df_test_data],axis=0).copy()

### 2.2. Funções referentes a lista

Função para avaliar a perfomance por lista

In [17]:
def performance_list(df_submission):  
  #Essa função necessita receber as submissões de uma turma ---------- depois podemos incrementar essa função ao ter uma turma associada e agrupar por elas
  #Me traz as porcentagens maximas de acerto por questão
  df_performance_list = df_submission.groupby(['user','list','question'])['hitPercentage'].max().reset_index()
  #Soma as porcentagens de uma lista unica
  df_performance_list = df_performance_list.groupby(['user','list'])['hitPercentage'].sum().reset_index()
  #Renomeando para melhorar entendimento
  df_performance_list.rename(columns={'hitPercentage':'totalHitPercentage'}, inplace = True)
  #Usando a função de dados de questão para ter informações sobre que questões estão associadas a lista
  df_question_data = question_data()
  #Agrupa por lista e questão para ter apenas uma questão de cada, a operação não importa muito, apenas existe para completar o agrupamento --------- pode ser uma boa trocar por drop_duplicates
  df_question_data = df_question_data.groupby(['list','question'])['id_list'].count().reset_index()
  #Conta quantas questões tem por lista
  df_question_data = df_question_data.groupby(['list'])['question'].count().reset_index()
  #Renomeando lista
  df_question_data.rename(columns={'question':'totalQuestionsList'}, inplace = True)
  #Merge
  df_performance_list = df_performance_list.merge(df_question_data, on = 'list')
  #Média de acerto por lista
  df_performance_list['mediaList'] = df_performance_list['totalHitPercentage'] / df_performance_list['totalQuestionsList']
  return df_performance_list

Função do gráfico de barras que, por lista, faz a contagem de quem fez mais e menos de 70% da lista. Necessita de passar a performance do aluno por lista

In [18]:
def graph_more_less_70_list_class(df_performance_list):
  #Aqui eu conto, por lista, quantos tiraram mais de 70% de acerto
  df_more70 = df_performance_list[df_performance_list['mediaList'] >= 70.0].groupby('list')['mediaList'].count().reset_index(name='more70')
  #E nesse conta quantos tiveram menos de 70%
  df_less70 = df_performance_list[df_performance_list['mediaList'] < 70.0].groupby('list')['mediaList'].count().reset_index(name='less70')
  #Aqui junta ambos os dataframes, usando o outer, que, se por acaso não tiver uma ocorrência de uma lista em um dos dataframes, mesmo assim eles vão ser adicionados, é união dos conjuntos
  df_less_more_70 = pd.merge(df_more70, df_less70, on = 'list', how = 'outer')
  #E, se caso acontecer a situação de um não existir, substituimos o nan por 0
  df_less_more_70.replace(np.NaN, 0, inplace = True)
  return df_less_more_70.to_json('df_less_more_70.json',force_ascii=False, orient='records') #############3 por hora essa função tamebm esta realizando downlaod

Função do grafico de distribuição de notas por lista. Necessita da performance do  

In [19]:
def graph_distribution_notes_list(df_performance_list):
  #Agrupa por lista, e conta as ocorrências
  return df_performance_list.groupby('list')['mediaList'].apply(lambda group: group.values.tolist()).to_json('df_distribuition_notes_list.json',force_ascii=False,orient='index')

In [20]:
def select_questions(df_question_data,listas_escolhidas,provas_escolhidas):
  #Selecionando onde List não é NaN
  df_prov1 = df_question_data.dropna(subset=['list'])#{}.fillna(0)
  #Selecionando onde Test não é NaN
  df_prov2 = df_question_data.dropna(subset=['test'])#.fillna(0)
  #Concatenação para permitir um loc que busque em todas as instancias
  pat1 = '|'.join(['({})'.format(re.escape(c)) for c in listas_escolhidas])
  pat2 = '|'.join(['({})'.format(re.escape(c)) for c in provas_escolhidas])
  #Busca
  df_prov1 = df_prov1.loc[df_prov1['list'].str.contains(pat1)].copy()
  df_prov2 = df_prov2.loc[df_prov2['test'].str.contains(pat2)].copy()
  return pd.concat([df_prov1, df_prov2],axis=0).copy()

### 2.3. Funções referentes a dificuldade

Função que faz o cálculo da performance dos alunos por dificuldade e retorna um dataframe. É necessário passar por parâmetro uma turma e uma variável bool com valor True(para Prova) ou False(para Lista)

In [21]:
def df_difficulty_performance (turma, prova = False):

  df_submissions = lop_submission(pd.DataFrame(turma, columns = ['id_class'])) # carregando dados de submissões da turma
  df_questions = select_questions(question_data(),listas_escolhidas,provas_escolhidas) # carregando dados de questões selecionadas
  
  if (prova): # filtro de provas
    df_questions = df_questions.dropna(subset=['test']).fillna(0)
  else:
    df_questions = df_questions.dropna(subset=['list']).fillna(0)

  df_geral = pd.merge(df_submissions,df_questions, on = "question", how = "inner") # juntando os dataframes de submissões e questões
  df_difficulty_performance = df_geral.groupby(['user','registration','question','difficulty'])['hitPercentage'].max().reset_index()
  df_difficulty_performance = df_difficulty_performance.groupby(['user','registration','difficulty'])['hitPercentage'].sum().reset_index()
  df_difficulty_performance.rename(columns={'hitPercentage':'totalHitPercentage'},inplace = True)

  df_questions = df_questions.groupby(['question','difficulty']).count().reset_index()
  df_questions = df_questions.groupby(['difficulty'])['question'].count().reset_index()
  df_questions.rename(columns={'question':'totalQuestions'},inplace=True)

  df_difficulty_performance = addData(df_difficulty_performance, df_difficulty_performance['difficulty'].unique(), df_difficulty_performance['user'].unique()  )
  df_difficulty_performance = df_difficulty_performance.merge(df_questions, on = 'difficulty', how = 'outer')
  df_difficulty_performance.replace(np.NaN, 0 , inplace = True)
  df_difficulty_performance['averageDifficulty'] = df_difficulty_performance['totalHitPercentage'] / df_difficulty_performance['totalQuestions']
  df_difficulty_performance = df_difficulty_performance.sort_values(['user','difficulty'])

  return df_difficulty_performance
  

Função que retorna o json de estudantes aprovados, reprovados e faltosos de todas as listas passadas pelo professor. É necessário passar por parâmetro uma turma, a nota esperada para aprovação e uma variável bool com valor True(para Prova) ou False(para Lista)

In [22]:
def graph_aprov (turma, nota_esperada, prova = False):

  #carregando os dados que serão utilizados na construção desse json
  df_class = lop_class()

  if (prova):
    performance_df = df_difficulty_performance(turma, True)
  else:
    performance_df = df_difficulty_performance(turma, False)

  #pegando da coluna 'averageDifficulty' do dataframe performance_df todos os que estão com media acima de 'nota_esperada'
  df_aprov = performance_df[performance_df['averageDifficulty']>=nota_esperada].groupby('difficulty')['averageDifficulty'].count().reset_index(name  = 'approved')
  #pegando da coluna 'averageDifficulty' do dataframe performance_df todos os que estão com media abaixo de 'nota_esperada'
  df_disaprov= performance_df[performance_df['averageDifficulty']<nota_esperada].groupby('difficulty')['averageDifficulty'].count().reset_index(name = 'disapproved')
  #juntando os dataframes criados acima em um só, com base na coluna 'difficulty' e com o método 'outer' para não haver perda de dados
  df_aprov_disaprov = pd.merge(df_aprov, df_disaprov, on='difficulty', how='outer')
  #como o método 'outer' foi utilizado é necessário substituir os vazios por zero
  df_aprov_disaprov.replace(np.NaN,0,inplace=True)
  #ordenando as colunas do dataframe
  df_aprov_disaprov = df_aprov_disaprov[['difficulty','approved','disapproved']]

  #buscando em df_class o numero de alunos matriculado na turma
  df_turma= df_class.query("id_class == {}".format(turma))[["id_class","studentsCount"]].drop_duplicates().reset_index().drop('index',axis=1)
  #armazenando em num_alunos o valor da primeira row de studentsCount, ou seja, o numero de alunos matriculados na turma
  num_alunos = int(df_turma["studentsCount"][0])
  #criando a coluna 'faltosos' sendo a diferença de num_alunos com a soma dos aprovados e reprovados
  df_aprov_disaprov['missing'] = num_alunos - (df_aprov_disaprov['approved'] + df_aprov_disaprov['disapproved'])

  #transformando todo o dataframe em inteiro
  df_aprov_disaprov = df_aprov_disaprov.astype(int)
  #ordenando o dataframe pela coluna dificuldade
  df_aprov_disaprov = df_aprov_disaprov.sort_values(by='difficulty')
  #renomeando as dificuldades para strings
  df_aprov_disaprov= df_aprov_disaprov.replace({'difficulty': {1:"Muito Fácil",2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} })
  return df_aprov_disaprov#.to_json('df_aprov_disaprov.json', force_ascii=False, orient='records')

Função que retorna o json com as médias de cada aluno por dificuldade e também a média da turma naquela dificuldade especifica. É necessário a passagem por parâmetro de uma turma e uma variável bool com valor True(para Prova) ou False(para Lista)

In [23]:
def averagebyDifficulty (turma, prova=False):
  if(prova):
    performance_df = df_difficulty_performance(turma, True) 
  else:
    performance_df = df_difficulty_performance(turma, False)

  performance_df.drop(['totalHitPercentage','totalQuestions'],axis=1,inplace=True)

  df_student_averagebyDifficulty = performance_df
  df_mean = df_student_averagebyDifficulty.groupby(['difficulty'])['averageDifficulty'].mean().reset_index()
  df_mean.rename(columns={'averageDifficulty':'classAverage'},inplace=True)
  df_student_averagebyDifficulty = df_student_averagebyDifficulty.merge(df_mean, how='outer', on='difficulty')
  df_student_averagebyDifficulty = df_student_averagebyDifficulty.sort_values(['user','difficulty'])
  df_student_averagebyDifficulty.replace({'difficulty': {1:"Muito Fácil",2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} }, inplace=True)
  return df_student_averagebyDifficulty#.to_json('df_student_averagebyDifficulty.json', force_ascii=False, orient='records')

Função que retorna o json com as média geral do aluno, referente a soma das médias que o aluno obteve em cada dificuldade sobre o número de dificuldades existentes. É necessário a passagem por parâmetro de uma turma e uma variável bool com valor True(para Prova) ou False(para Lista)

In [24]:
def averageAllDifficulty (turma, prova = False ):
    
    if (prova):
        performance_df = df_difficulty_performance(turma, True)
    else:
        performance_df = df_difficulty_performance(turma, False)

    performance_df.drop(['totalHitPercentage','totalQuestions'],axis=1,inplace=True)

    df_allmean = performance_df.groupby(['user','registration','difficulty','averageDifficulty']).count().reset_index()
    df_allmean = df_allmean.groupby(['user','registration'])['averageDifficulty'].mean().reset_index()
    df_allmean.rename(columns={'averageDifficulty':'averageAllDifficulty'},inplace=True)
    return df_allmean#.to_json('df_averageAllDifficulty.json', force_ascii=False, orient='records')

### FUNÇÕES TEMPORÁRIAS

Função para adicionar dados faltoso ao dataframe de performance por dificuldade

In [25]:
def addData (df_difficulty_performance, difficulties, students):
  new_obs = pd.DataFrame()

  for i in df_difficulty_performance.columns.tolist():
    new_obs[i]= [np.nan]
  
  for student in students:
    new_obs["user"] = [student]
    df_student = df_difficulty_performance[df_difficulty_performance["user"] == student].reset_index().drop('index', axis=1)
    diff_questions = list(set(df_student["difficulty"].unique()) ^ set(difficulties))
    
    if len(diff_questions)>0:
      try:
        registration = df_student["registration"].iloc[0]
      except:
        break
      for difficulty in diff_questions:
        new_obs["difficulty"] = [difficulty]
        new_obs["registration"] = [registration]
        df_difficulty_performance = df_difficulty_performance.append(new_obs, ignore_index = True)
  return df_difficulty_performance

Função de média ponderada por dificuldade

In [26]:
# 1: Assume-se que a coluna numérica de interesse p/ calcular a média é sempre a última, e não pode haver mais de uma coluna numérica de média no mesmo df.
# 2: Assume-se que a variável "ordered_dfs_list" terá todos os seus dataframes se tratando da mesma coisa, e com as mesmas colunas.
# 3: Deve-se chamar a função da seguinte forma: graphWeightedAverage([df1.copy(),df2.copy()]). Caso não seja atribuido uma lista de pesos (ordered_weight_list = [x,y,z], se faz uma média aritmética.)

def graphWeightedAverage(ordered_dfs_list,ordered_weight_list=None,subjects=subjects.copy()):

  number_of_dfs = len(ordered_dfs_list) # calcula o número de dfs passados em ordered_dfs_list

  initial_column_names = ordered_dfs_list[0].columns.tolist() # armazena o nome das colunas no df antes de manipular
  last_column_name = initial_column_names[-1] # armazena o nome da última coluna (que é a numérica de interesse p/ calcular a média)

  # Etapa para checar se tá tudo ok com os pesos, e atribuir um valor default caso não tenha sido especificado
  if ordered_weight_list == None:
    ordered_weight_list = [0.5]*number_of_dfs # default
  elif number_of_dfs != len(ordered_weight_list):
      raise ValueError('Both lists must have the same length')
      return

  for i in range(0,number_of_dfs): # varrendo todos os dfs da lista (um for bem pequeno. Atualmente, apenas 2 loops)
    if 'registration' in ordered_dfs_list[i].columns.tolist(): # etapa necessária para lidar com o problema do lop de conseguir inserir caractere na matricula
      ordered_dfs_list[i]["registration"] = ordered_dfs_list[i]["registration"].astype(str)

    #ordered_dfs_list[i]["df_id"] = i
    ordered_dfs_list[i]["weight"] = ordered_weight_list[i] # atribui o respectivo peso para cada linha do df

    if i < 1: # concatena todos os dfs da lista
      df = ordered_dfs_list[0]
    else:
      df = pd.concat([df,ordered_dfs_list[i]])


  df[last_column_name] = df[last_column_name] * df["weight"] # multiplica a coluna de interesse pelo peso

  #as etapas a seguir são necessárias para eliminar o problema de haver assuntos na lista de exercícios e não haver na lista de provas, por exemplo.
  #desse modo, caso exista em um e não no outro, ignora-se o peso do elemento faltante no cálculo da média.
  student_weight_list = df.groupby(initial_column_names[:-1])["weight"].sum().reset_index().iloc[:,-1] 
  df = df.groupby(initial_column_names[:-1])[last_column_name].sum().reset_index()
  df[last_column_name] = df[last_column_name]/student_weight_list

  return df#.to_json("df_weighted_average_all_subjects.json",force_ascii=False, orient='records')

NameError: name 'subjects' is not defined

## 3. Leitura da Base
<a id='leiturabase'></a>
### 3.1. Leitura de endpoints e key

In [27]:
key = read_txt('key.txt')
endpoint_teacher = read_txt('endpoint_teacher.txt')
endpoint_class = read_txt('endpoint_class.txt')
endpoint_submission = read_txt('endpoint_submission.txt')
endpoint_question = read_txt('endpoint_question.txt')

## 4. Sandbox 

Nessa seção é feito uma exploração dos dados para criação de funções

In [121]:
listas_escolhidas = ['Laboratório 01 - Expressões Aritméticas',

'Aula - Introdução Expressões Aritiméticas',

'Aula - Funções Aritméticas',

'Treinamento - Expressões Aritméticas',

'(Lop) Lista de Laboratório 2 - Estrutura de decisão',

'(Lop) Estruturas de decisão - problemas sobre divisibilidade',

'(Lop) Estruturas de decisão - Múltiplas decisões',

'(Lop) Estruturas de decisão - Operadores Lógicos',

'Repetição condicional - Lista Resolvida (LOP)',

'Repetição condicional - Lista Prática (LOP)',

'Repetição condicional - Lista de Exercícios (LOP)',

'Repetição contada - Lista Resolvida (LOP)',

'Repetição contada - Lista Prática (LOP)',

'Repetição contada - Lista de Exercícios (LOP)',

'Vetores - Lista Resolvida (LOP)',

'Vetores - Lista de Exercícios (LOP)',

'Vetores - Lista Prática (LOP)']

provas_escolhidas = ['Prova LoP - Turma 05 - 2020.6']

In [122]:
turma = ['f2dd7bef-5b5d-4cb3-9efa-aa8652af0605']

In [123]:
df_submission = lop_submission(pd.DataFrame(turma, columns = ['id_class']))

In [124]:
#df_submission

In [125]:
df_questions = select_questions(question_data(),listas_escolhidas,provas_escolhidas)

In [126]:
#df_questions

In [127]:
df_questionsCount = df_questions.groupby(['question','list']).count().reset_index()
df_questionsCount = df_questionsCount.groupby(['list'])['question'].count().reset_index()

In [128]:
#df_questionsCount

In [129]:
df_submission = df_submission.merge(df_questions[:][['question','difficulty','tag1']], on = 'question', how = 'inner')
df_submission[:]['tag1'].replace(np.NaN, 'ASSUNTO_EM_BRANCO', inplace= True)

In [130]:
df_submission['timeConsumingInMinutes'] = df_submission['timeConsuming']/(60*1000)

In [131]:
#df_submission[df_submission['timeConsumingInMinutes']>0].sort_values(by='timeConsuming')

Pegando a data de submissão

In [132]:
from datetime import datetime

In [133]:
df_submission['createdAt'] = pd.to_datetime(df_submission['createdAt'])

In [134]:
df_submission['dateSubmission'] = df_submission['createdAt'].dt.date

In [135]:
#df_submission

In [136]:
colunas = ['difficulty','question','dateSubmission']
df_teste = df_submission[df_submission.timeConsuming>0][colunas].groupby(['question','difficulty']).nunique().reset_index()

In [137]:
#df_teste['meanDaysPerQuestion'] = round(df_teste['dateSubmission'] / df_teste ['question'],2)

In [138]:
#df_teste.sort_values(by='difficulty')

In [139]:
#df_submission['dateSubmission'].drop_duplicates().value_counts().reset_index()

In [140]:
colunas = ['list','timeConsumingInMinutes','test']
df_timeMean = df_submission[df_submission.timeConsumingInMinutes>0][colunas].groupby(['list']).mean().reset_index()
df_timeMean2 = df_submission[df_submission.timeConsumingInMinutes>0][colunas].groupby(['test']).mean().reset_index()

In [141]:
df_timeMean['timeConsumingInMinutes'] = round(df_timeMean['timeConsumingInMinutes'],2)
#df_timeMean

In [142]:
df_timeMean2['timeConsumingInMinutes'] = round(df_timeMean2['timeConsumingInMinutes'],2)
#df_timeMean2

In [143]:
#round(df_timeMean['timeConsumingInMinutes'].mean(),2)

In [144]:
colunasDif = ['timeConsumingInMinutes','difficulty']
df_timeAveragePerDifficulty = df_submission[df_submission.timeConsumingInMinutes>0][colunasDif].groupby(['difficulty']).mean().reset_index()
df_timeAveragePerDifficulty = df_timeAveragePerDifficulty.replace({'difficulty': {1:"Muito Fácil", 2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} })
df_timeAveragePerDifficulty = round(df_timeAveragePerDifficulty,2)
#df_timeAveragePerDifficulty

In [145]:
colunasSubjects = ['timeConsumingInMinutes','tag1']
df_timeAveragePerSubject = df_submission[df_submission.timeConsumingInMinutes>0][colunasSubjects].groupby(['tag1']).mean().reset_index()
#df_timeAveragePerDifficulty.replace( 0, 'ASSUNTO_EM_BRANCO', inplace=True )
#df_timeAveragePerDifficulty = df_timeAveragePerDifficulty.replace({'difficulty': {1:"Muito Fácil", 2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} })
df_timeAveragePerSubject = round(df_timeAveragePerSubject,2)
#df_timeAveragePerSubject